## Import 

In [1]:
from kevin_utils2 import *
from keras.layers.merge import dot, add, concatenate
from __future__ import division, print_function
import gzip, os
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder


Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

In [3]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

In [4]:
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [5]:
df = getDF('reviews_Home_and_Kitchen_5.json.gz')

In [6]:
df=df.dropna(axis=1,how='any')


In [7]:
df=df.head(n=10000)


In [8]:
#c={
#'1.0':'One',
#'2.0':'Two',
#'3.0':'Three',
#'4.0':'Four',
#'5.0':'Five'
#}

In [9]:
#df['overall'] = df['overall'].astype(str)
#df['overall']=df.overall.map(c)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 8 columns):
reviewerID        10000 non-null object
asin              10000 non-null object
helpful           10000 non-null object
unixReviewTime    10000 non-null int64
reviewText        10000 non-null object
overall           10000 non-null float64
reviewTime        10000 non-null object
summary           10000 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 703.1+ KB


In [11]:
X = df.reviewText
y = df.overall

## Setup Data

In [12]:
##encode class values as integers
#encoder = LabelEncoder()
#encoder.fit(y)
#encoded_Y = encoder.transform(y)
# #convert integers to dummy variables (i.e. one hot encoded)
#y_10 = np_utils.to_categorical(encoded_Y)

In [13]:
from keras.utils import to_categorical

y_10 = to_categorical(y)


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_10, test_size=0.2, random_state=42)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(1,1), token_pattern=r'\b\w{1,}\b')

In [16]:
vect.fit(X_train)
vocab = vect.vocabulary_

In [17]:
def convert_X_to_X_word_ids(X):
    return X.apply( lambda x: [vocab[w] for w in [w.lower().strip() for w in x.split()] if w in vocab] )

In [18]:
X_train_word_ids = convert_X_to_X_word_ids(X_train)
X_test_word_ids  = convert_X_to_X_word_ids(X_test)

In [19]:
type(X_train)

pandas.core.series.Series

In [20]:
X_train_word_ids=X_train_word_ids.tolist()
X_test_word_ids=X_test_word_ids.tolist()
#y_train=y_train.tolist()
#y_test=y_test.tolist()

In [21]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in X_train_word_ids]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in X_test_word_ids]

In [22]:
lens = np.array(map(len, trn))
(lens.max(), lens.min(), lens.mean())

(1334, 0, 75.086500000000001)

In [23]:
seq_len = 100

trn = pad_sequences(trn, maxlen=seq_len, value=0)
test = pad_sequences(test, maxlen=seq_len, value=0)

In [24]:
trn.shape,test.shape


((8000, 100), (2000, 100))

## Single hidden layer NN

In [46]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(6, activation='sigmoid')])

In [47]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 32)           160000    
_________________________________________________________________
flatten_4 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 100)               320100    
_________________________________________________________________
dropout_10 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 6)                 606       
Total params: 480,706
Trainable params: 480,706
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.fit(trn, y_train, validation_data=(test, y_test), nb_epoch=4, batch_size=64)


Train on 8000 samples, validate on 2000 samples
Epoch 1/4
8000/8000 [==============================] - 1s 111us/step - loss: 1.1580 - acc: 0.6840 - val_loss: 0.9667 - val_acc: 0.6935
Epoch 2/4
8000/8000 [==============================] - 1s 72us/step - loss: 1.0308 - acc: 0.6897 - val_loss: 0.9290 - val_acc: 0.6935
Epoch 3/4
8000/8000 [==============================] - 1s 73us/step - loss: 0.9657 - acc: 0.6897 - val_loss: 0.9070 - val_acc: 0.6935
Epoch 4/4
8000/8000 [==============================] - 1s 72us/step - loss: 0.9158 - acc: 0.6906 - val_loss: 0.9020 - val_acc: 0.6940


## CNN, 1D

In [43]:
conv1 = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, dropout=0.2),
    Dropout(0.2),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(6, activation='sigmoid')])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 5, padding="same", activation="relu")`
  after removing the cwd from sys.path.


In [44]:
conv1.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])


In [45]:
conv1.fit(trn, y_train, validation_data=(test, y_test), nb_epoch=10, batch_size=64)


Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 145us/step - loss: 1.0505 - acc: 0.6846 - val_loss: 0.9994 - val_acc: 0.6935
Epoch 2/10
8000/8000 [==============================] - 1s 107us/step - loss: 0.9586 - acc: 0.6897 - val_loss: 0.9447 - val_acc: 0.6935
Epoch 3/10
8000/8000 [==============================] - 1s 108us/step - loss: 0.9145 - acc: 0.6897 - val_loss: 0.9330 - val_acc: 0.6935
Epoch 4/10
8000/8000 [==============================] - 1s 106us/step - loss: 0.8674 - acc: 0.6897 - val_loss: 0.9207 - val_acc: 0.6950
Epoch 5/10
8000/8000 [==============================] - 1s 107us/step - loss: 0.8326 - acc: 0.6939 - val_loss: 0.9390 - val_acc: 0.6920
Epoch 6/10
8000/8000 [==============================] - 1s 107us/step - loss: 0.7968 - acc: 0.6996 - val_loss: 0.9563 - val_acc: 0.6895
Epoch 7/10
8000/8000 [==============================] - 1s 109us/step - loss: 0.7670 - acc: 0.7017 - val_loss: 0.9679 - val_acc: 0.6915


In [42]:
#conv1.save_weights('conv1')


In [28]:
#conv1.load_weights('conv1')


## LSTM

In [25]:
rnn = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, mask_zero=True,
              W_regularizer=l2(1e-6), dropout=0.2),
    LSTM(100, consume_less='gpu'),
    Dense(6, activation='sigmoid')])
rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
rnn.summary()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(5000, 32, mask_zero=True, embeddings_regularizer=<keras.reg..., input_length=100)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(100, implementation=2)`
  after removing the cwd from sys.path.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 606       
Total params: 213,806
Trainable params: 213,806
Non-trainable params: 0
_________________________________________________________________


In [27]:
rnn.fit(trn, y_train, validation_data=(test, y_test), nb_epoch=5, batch_size=64)


/usr/local/lib/python2.7/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 42s 5ms/step - loss: 0.3166 - acc: 0.8957 - val_loss: 0.2721 - val_acc: 0.8978
Epoch 2/5
8000/8000 [==============================] - 39s 5ms/step - loss: 0.2753 - acc: 0.8966 - val_loss: 0.2716 - val_acc: 0.8978
Epoch 3/5
8000/8000 [==============================] - 39s 5ms/step - loss: 0.2738 - acc: 0.8966 - val_loss: 0.2697 - val_acc: 0.8978
Epoch 4/5
8000/8000 [==============================] - 39s 5ms/step - loss: 0.2711 - acc: 0.8966 - val_loss: 0.2636 - val_acc: 0.8981
Epoch 5/5
8000/8000 [==============================] - 39s 5ms/step - loss: 0.2622 - acc: 0.8986 - val_loss: 0.2627 - val_acc: 0.8988


## Model Predict

In [49]:
prediction = rnn.predict(np.array(test))
prediction2 = model.predict(np.array(test))
prediction3 = conv1.predict(np.array(test))


## Evaluate The Model With k-Fold Cross Validation

In [31]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [34]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [35]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [46]:
results = cross_val_score(prediction, X_test, y_test, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

TypeError: estimator should be an estimator implementing 'fit' method, array([[  4.33578761e-03,   1.11002535e-01,   1.18002132e-01,
          2.23158062e-01,   3.57010156e-01,   9.33912754e-01],
       [  3.21856714e-05,   3.46616306e-03,   4.58798464e-03,
          7.65424687e-03,   1.02789648e-01,   9.99610841e-01],
       [  1.03913648e-04,   9.30281263e-03,   1.27427103e-02,
          2.06015855e-02,   1.39002904e-01,   9.98331964e-01],
       [  2.05472996e-03,   5.04694209e-02,   5.91314360e-02,
          8.46662894e-02,   3.28209400e-01,   9.81742501e-01],
       [  1.94137986e-03,   4.86882105e-02,   6.96897283e-02,
          9.78604779e-02,   2.87750483e-01,   9.83010888e-01],
       [  1.80258154e-04,   1.24613484e-02,   1.28406817e-02,
          2.37544216e-02,   1.48694307e-01,   9.97727454e-01],
       [  1.20217446e-02,   1.28007978e-01,   1.89693868e-01,
          2.63502955e-01,   4.00781810e-01,   9.31031406e-01],
       [  1.20968925e-05,   2.31395010e-03,   3.217227

In [42]:
winner = np.argwhere(prediction[i] == np.amax(prediction[i]))

numpy.ndarray

In [43]:
for i in ranger:
    
    inverted = np.argwhere(prediction[i] == np.amax(prediction[i]))
    inverted2 = np.argwhere(y_test[i] == np.amax(y_test[i]))

In [48]:
type(inverted)

numpy.ndarray

## Print Results

In [1]:
from numpy import argmax

ranger = range(0,1000)

for i in ranger:
    
    inverted = argmax(prediction[i])
    inverted2 = argmax(y_test[i])
    
    print(X_test.values[i])
    print('pred_class:',(inverted))
    print('true_class:', (inverted2))

NameError: name 'prediction' is not defined

In [34]:
ranger = range(0,1000)

for i in ranger:
    
    inverted = argmax(prediction[i])

    print('pred_class:',(inverted))

pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_class: 5
pred_c

In [50]:
for i in ranger:
    
    inverted = argmax(prediction2[i])
    inverted2 = argmax(y_test[i])
    
    print(X_test.values[i])
    print('pred_class single:',(inverted))
    print('true_class:', (inverted2))

Great Product for taco lovers. The design is nice with the ability to bake 4 tacos a time. The only draw back that I had hence the 4 stars instead of 5 is the non-stick coating. It has it's plus side with the easy clean-up, but the downside I had was the Tacos kept sliding off. I did find a solution for me by laying the taco on Al Foil with a 1/2 inch hem at both ends then hanging them over the peaks. It works for me because I wanted the taco shape without frying them (trying to control my cholesterol). Glad I bought it and would again.
pred_class single: 5
true_class: 4
Easy Grip for stability, allowed cutting harder vegetables more easily and alsokept up the speed of chopping, I recommend this item for your everyday chopping.
pred_class single: 5
true_class: 4
This peeler really is great, it peels things so smoothly and effortlessly, and the grips on the sides are durable and help your hands from slipping or sliding off.  So happy with this purchase.  All of my roommates love it too!

good power and easy to maneuver compactaccessories are a bit flimsy -deserves 3 3/4 stars because of flimsy accessories.reasonable price
pred_class single: 5
true_class: 4
I bought this based on the reviews and was disappointed to find that it was extremely dull.  I can't believe how dull it is after such good reviews.  Do you all just happy with dull knives or did I just get a bad knife?  I bought an electric sharpener and may keep the knife if I get a good edge on it, but I shouldn't have to sharpen a brand new knife.  I know this isn't as high quality as others in the Henckels line but I'm tired of shopping and spending.  I may return it for a better one.Update:  This is a worthless knife.  I've tried sharpening it and it still has a hard time going through veggies...it's wider than most knives and I think alot of the resistance is due to the width of the blade.  Bad knife, don't buy.  I don't know why it has all the good reviews.
pred_class single: 5
true_class: 1
I am a 62 year ol

In [51]:
for i in ranger:
    
    inverted = argmax(prediction3[i])
    inverted2 = argmax(y_test[i])
    
    print(X_test.values[i])
    print('pred_class conv:',(inverted))
    print('true_class:', (inverted2))

Great Product for taco lovers. The design is nice with the ability to bake 4 tacos a time. The only draw back that I had hence the 4 stars instead of 5 is the non-stick coating. It has it's plus side with the easy clean-up, but the downside I had was the Tacos kept sliding off. I did find a solution for me by laying the taco on Al Foil with a 1/2 inch hem at both ends then hanging them over the peaks. It works for me because I wanted the taco shape without frying them (trying to control my cholesterol). Glad I bought it and would again.
pred_class conv: 5
true_class: 4
Easy Grip for stability, allowed cutting harder vegetables more easily and alsokept up the speed of chopping, I recommend this item for your everyday chopping.
pred_class conv: 5
true_class: 4
This peeler really is great, it peels things so smoothly and effortlessly, and the grips on the sides are durable and help your hands from slipping or sliding off.  So happy with this purchase.  All of my roommates love it too!
pre

pred_class conv: 5
true_class: 5
Bought this opener based on reviews. It works well but it does 'hang up' on cans if there is even the slightest dent. I either have to release the grip and reposition the opener or use a different can opener altogether - definitely a pain...  When it does work perfectly it is smooth and easy to operate whereas my other hand openers (I have 2 others) take a bit more effort.  If it worked perfectly I'd be singing its praises.  But - had I known it was going to have this issue I wouldn't have bought it - the slight increase in effort I have to use with my other openers wins over the 'frustration factor' of this one - not to mention at $17.99 it is much more expensive than any opener you can buy at your grocery store or Walmart.  Lately I find I'm reaching for one of my old openers when I need to open a can.
pred_class conv: 4
true_class: 3
I'm an OXO fan to begin with so that's why I ordered this little brush.  It not only scrubs veggies well, it fits nice

These are quite honestly the best measuring cups I have owned to date.  Unlike the others with problems having the handle bend, mine have been unaffected.  Perhaps its a simple solution of gripping more towards the 'cup' than at the far end of the handle, but not too much to ask to try and prevent the 'bending' from happening.  As with all OXO products, they are very well made, have a great non slip handle, and beleive it or not, having the sizes written in bright differing colors helps.  Overall very satisfied and would highly recommend.
pred_class conv: 5
true_class: 5
This smaller pan works for me.  It is just the right size for a medium cheesecake!  the sides are nice and high, the non-stick coating is useful and the seal is tight.  Excellent value.
pred_class conv: 5
true_class: 5
This is one of my favorite spatulas. It has a nice long handle with a comfortable rubber grip. The spatula has a broad surface, the better to flip things that might otherwise fall apart. It doesn't scrat